In [2]:
!git clone https://github.com/tutorinedutech/tutorin-ml.git

fatal: destination path 'tutorin-ml' already exists and is not an empty directory.


In [4]:
import sys
import os
sys.path.append('/kaggle/working/tutorin-ml')
from src.model.siamese import SiameseModel
from src.data.data_loader import DataLoader, Preprocess
from src.utils.helper_function import load_data

2024-06-02 03:33:48.831223: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-02 03:33:48.831343: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-02 03:33:48.994622: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
import tensorflow as tf

In [23]:
#PARAMETER
data_path = '/kaggle/working/tutorin-ml/data_source/processed/clean_data.csv'
TRAINING_SPLIT = 0.7
BUFFER_SIZE = 42

TRAIN_BATCH_SIZE = 8
TRAIN_NUM_BATCHES = 20

VAL_BATCH_SIZE = 8
VAL_NUM_BATCHES = 20

TEST_BATCH_SIZE = 8
TEST_NUM_BATCHES = 20

AUTOTUNE = tf.data.experimental.AUTOTUNE

In [24]:
df = load_data(data_path)
df.head()

,Unnamed: 0,Prompt,combined,Skor Label
0,0,saya ingin belajar fisika dengan mahasiswa fis...,jenis kelamin : laki-laki provinsi : daerah is...,5.0
1,1,tolong carikan tutor fisika dan matematika den...,jenis kelamin : laki-laki provinsi : daerah is...,4.0
2,2,butuh tutor fisika buat anak sma dengan cara m...,jenis kelamin : perempuan provinsi : daerah is...,0.0
3,3,carikan saya guru privat tentang hukum karena ...,jenis kelamin : perempuan provinsi : daerah is...,4.0
4,4,guru dengan tegas dan semangat mengajar anak s...,jenis kelamin : laki-laki provinsi : daerah is...,2.0


In [25]:
data_loader = DataLoader(df, train_split=TRAINING_SPLIT, buffer_size=BUFFER_SIZE)
dataset = data_loader.get_tensor_data()
dataset = data_loader.get_prefetch_data(dataset)
dataset_train, dataset_val, dataset_test = data_loader.split_data(dataset)

In [26]:
train_data = Preprocess(dataset_train, TRAIN_BATCH_SIZE, TRAIN_NUM_BATCHES, AUTOTUNE)
val_data = Preprocess(dataset_val, VAL_BATCH_SIZE, VAL_NUM_BATCHES, AUTOTUNE)
test_data = Preprocess(dataset_test, TEST_BATCH_SIZE, TEST_NUM_BATCHES, AUTOTUNE)

train_data = train_data.preprocessing_data()
val_data = val_data.preprocessing_data()
test_data = test_data.preprocessing_data()

In [27]:
siamese_model = SiameseModel()

Attaching 'metadata.json' from model 'keras/roberta/keras/roberta_base_en/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/roberta/keras/roberta_base_en/2' to your Kaggle notebook...
Attaching 'preprocessor.json' from model 'keras/roberta/keras/roberta_base_en/2' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/roberta/keras/roberta_base_en/2' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/roberta/keras/roberta_base_en/2' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.json' from model 'keras/roberta/keras/roberta_base_en/2' to your Kaggle notebook...
Attaching 'assets/tokenizer/merges.txt' from model 'keras/roberta/keras/roberta_base_en/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/roberta/keras/roberta_base_en/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/roberta/keras/roberta_base_en/2' to your Kaggle notebook...
Attaching 'config.json' f

In [29]:
siamese_model.model.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ sentences           │ (None, 2)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ split_1 (Split)     │ [(None, 1),       │          0 │ sentences[0][0]   │
│                     │ (None, 1)]        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_5        │ (None, 768)       │ 124,052,7… │ split_1[0][0],    │
│ (Functional)        │                   │            │ split_1[0][1]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot_1 (Dot)         │ (None, 1)         │          0 │ functional_5[0][… │
│                     │                   │            │ functional_5[1][… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 124,052,736 (473.22 MB)

 Trainable params: 124,052,736 (473.22 MB)

 Non-trainable params: 0 (0.00 B)

In [30]:
sentences = [
    "Saya suka mengajar bisnis edutech dan pemanfaatan AI di jakarta",
    "Saya senang mengajar fisika dan sudah berpengalaman di Yogyakarta"
]
query = ["Mencari tutor fisika di Yogyakarta."]

encoder = siamese_model.get_encoder()

sentence_embeddings = encoder(tf.constant(sentences))
query_embedding = encoder(tf.constant(query))

cosine_simalarities = tf.matmul(query_embedding, tf.transpose(sentence_embeddings))
for i, sim in enumerate(cosine_simalarities[0]):
    print(f"cosine similarity between sentence {i+1} and the query = {sim} ")

cosine similarity between sentence 1 and the query = 0.9664742350578308 
cosine similarity between sentence 2 and the query = 0.9698975086212158 


In [31]:
siamese_model.model.compile(
    loss=tf.keras.losses.MeanSquaredError(),
    optimizer=tf.keras.optimizers.Adam(2e-5),
    jit_compile=False,
)

In [32]:
siamese_model.model.fit(train_data, validation_data=val_data, epochs=3)

Epoch 1/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 88s 88s/step - loss: 1.4107 - val_loss: 3.0963
Epoch 2/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 660ms/step - loss: 2.0421 - val_loss: 1.8259
Epoch 3/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 633ms/step - loss: 1.8693 - val_loss: 0.7170


In [33]:
sentences = [
    "Saya suka mengajar bisnis edutech dan pemanfaatan AI di jakarta",
    "Saya senang mengajar fisika dan sudah berpengalaman di Yogyakarta"
]
query = ["Mencari tutor fisika di Yogyakarta."]

encoder = siamese_model.get_encoder()

sentence_embeddings = encoder(tf.constant(sentences))
query_embedding = encoder(tf.constant(query))

cosine_simalarities = tf.matmul(query_embedding, tf.transpose(sentence_embeddings))
for i, sim in enumerate(cosine_simalarities[0]):
    print(f"cosine similarity between sentence {i+1} and the query = {sim} ")

cosine similarity between sentence 1 and the query = 0.8720117211341858 
cosine similarity between sentence 2 and the query = 0.874594509601593 
